## BrainTriage Data Visualisation and Exploration

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import SimpleITK as sitk
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math as math
import csv
import pandas as pd
from tqdm.notebook import tqdm
import pdb
from torchvision import models
import random
import os
import sys
## reload scripts before executing them
%load_ext autoreload
%autoreload 2
sys.path.append(".")
sys.path.append("..")
from visualisation.slice_plotter import plot_slices, plot_slices_by_acquisition, compare_slices
from visualisation.blob_plotter import get_brain, bright_blobs_plotter


In [ ]:
in_dir = '../../data/full'
out_dir = '../../data_sliced'


## Interesting 

Things to consider.
TE = echo time
TR = repetition time



![image](../images/differences.png)


T1, T2, and T2-Flair show  some characterics in an opposing way. White matter is light in T1 and dark gray in T2 and T2 Flair


We can differentiate between CSF, WhiteMatter, Gray Matter Fat and Inflammations. 
We think it will be interesting to start with visualizations of slices of patients with vs without abnormalities. The next step would be to find the brightest spots since these could indicate inflammation in T2 and T2-FLAIR slices.  

### Hyperintensity
One interesting indicator could be [hyperintensity](https://en.wikipedia.org/wiki/Hyperintensity#:~:text=Hyperintensities%20refer%20to%20areas%20of,by%20demyelination%20and%20axonal%20loss.), which are ares with high intensity. These areas can indicate ischemia or bleedings, e.g. abnormalities. 

### Symmetry / Asymmetry
If we find something that we consider an artifact it might be interesting to check if the artifact occurs in both hemispheres. If that is the case then it might not be the actual abnormality. 


However, we need to be careful with these assumptions since we do not have enough experience to diagnose/identify abnormalities. These assumptions are mere tools to find probable abnormalities in slices.

In [ ]:
normal_patients_dir = os.path.join(in_dir , 'normal')
normal_patient_numbers = os.listdir(normal_patients_dir)
print("Total nr of normal patient indices {:d} ".format(len(normal_patient_numbers)))
abnormal_patients_dir = os.path.join(in_dir , 'abnormal')
abnormal_patient_numbers = os.listdir(abnormal_patients_dir)
print("Total nr of abnormal patient indices {:d} ".format(len(abnormal_patient_numbers)))


In [ ]:
## generate random patient numbers from normal and abnormal 
## determine number of slices
## one slice, range (tuple) or select number of slices ,e.g. a list

normal_patients = random.sample(normal_patient_numbers, k = 20)
abnormal_patients = random.sample(abnormal_patient_numbers, k = 20)


In [ ]:
options = ['single', 'list', 'range']
option = random.sample(options, k = 1)[0]


In [ ]:
def generate_slice_numbers(data_type):
    slices = list(range(0,32))
    ## if single then 1 slice
    if (data_type == 'single'):
        slice_number = random.choice(slices)
        return slice_number
    elif (data_type == 'range'):
        range_start = random.choice(range(0,30))
        range_limit = range_start + 5 if range_start+5 < 30 else range_start +  np.abs((range_start) - 31)

        range_end = random.choice(range(range_start+1, range_limit+1))
        return (range_start,range_end)
    else:
        number_of_slices = random.choice(range(1,6))
        indices = random.sample(slices, k=number_of_slices)
        return indices



In [ ]:

for normal,abnormal in zip(normal_patients[:10], abnormal_patients[:10]):
    options = ['single', 'list', 'range']
    option = random.sample(options, k = 1)[0]
    slices = generate_slice_numbers('range')
    compare_slices(normal,abnormal,slices,in_dir)


## Abnormality Inspection

We would like to have a closer look at some of these brains. We are goinng to inspect the following patients more closely;
    * abnormal 340
        * interesting bright spots
    * abnormal 891 
    * abnormal 580
    * abnormal 849
    * abnormal 2002 and normal 25
        * the normal slice (1) have some black areas
    * abnormal 1126 and normal 1492
        * the normal slice (29,31) have the same blak areas.

Starting from slice 3 we can see a bright spot apppearing on the left which gets bigger from slice 3 to 5 and disappears - it is asymmetrical. This spot is bright on T2 FLAIR and T2 and a bit darker on T1, it could be an inflammation. 

Starting from slice 15 to 23 we can see some irregular bright spots again on both T2 and T2 Flair


In [ ]:
plot_slices_by_acquisition(340,"abnormal",(0,31),in_dir)

Next, we are going to plot the slices of interest and inspect them more closely.
We plot circles around areas with high intensities, in some plots these areas are the outlining, we can ignore these circles. Slices 15, 18, 19 and 21 indicate the brighter areas which could be inflammations or abnormalities.

In [ ]:
slices_of_interest = list(range(3,6)) + list(range(15,24))
bright_blobs_plotter(340,"abnormal",slices_of_interest,in_dir)

slices 17 - 18 shows a bright spot in a what we assume to be a ventrikel

slices 19 -21 shows bright spot with inner black on T2 flair, this same spot is a bright spot
on T2 and a darker spot on T1

In [ ]:
plot_slices_by_acquisition(891,"abnormal",(0,31),in_dir)

Lets inspect these slices again.

In [ ]:
slices_of_interest = list(range(17,18)) + list(range(19,21))
bright_blobs_plotter(891,"abnormal",slices_of_interest,in_dir)

The following four normal brains (1642, 580, 25 and 1492) are plotted to inspect the first two and last two slices which some interesting black areas. However, since these are symmetric for we assume this is normal. These patterns are also observed in patient 849 and 20002, which have abnormalities in the brain. Slice 10 and 11 of patient 849 also show bright spots in T2FLAIR acquisition which could also be an abnormality. We inspected these three patients because of the patterns in the first and last two slices. These pattern were observable in both normal and abnormal patients. However, after inspecting an abnormal patient (849) which had these patterns we found other interesting pattern in slices 16 - 22. It is more likely that the black areas are normal while the patterns ins slices 16 - 22 in 849 are abnormal. 

To conclude, we found some interesting black areas in normal and abnormal brains. We don't know exactly what it is but since they exist in normal brains we assume that they are not abnormalities. 

In [ ]:
plot_slices_by_acquisition(1642,"normal",(0,31),in_dir)

In [ ]:
plot_slices_by_acquisition(580,"abnormal",(0,31),in_dir)

In [ ]:
plot_slices_by_acquisition(25,"normal",(0,31),in_dir)

In [ ]:
plot_slices_by_acquisition(1492,"normal",(0,31),in_dir)

In [ ]:
plot_slices_by_acquisition(849,"abnormal",(0,31),in_dir)

In [ ]:
plot_slices_by_acquisition(2002,"abnormal",(0,31),in_dir)

In [ ]:
plot_slices_by_acquisition(1126,"abnormal",(0,31),in_dir)

In [ ]:

for abnormal in abnormal_patients:
    options = ['single', 'list', 'range']
    option = random.sample(options, k = 1)[0]
    slices = generate_slice_numbers('range')
#     compare_slices(normal,abnormal,slices,in_dir)
    bright_blobs_plotter(abnormal,"abnormal",slices,in_dir)

After looking at the slices in the cells above and plotting interesting slices as well as circling interesting spots. We think that it might be interesting to see if the average intensity of the acquisitions are different between normal and abnormal brains. Ou reasoning is that in both T2 and T2 flair the inflammations have a light colour whereas in T1 inflammations have darker colour. Since inflammations are types of abnormalities we think that the verage intensities in T2 and T2 FLAIR will be higher in abnormal patients compared to normal patients. If this happens to be the case then it might be interesting to perform augmentations like gaussian smoothing. We will plot and inspect histograms of the average intensities for each acquisition of all patients over sets of slices, based ont these histograms we are hoping to draw conclusions that will guide us in our data preprocessing/augmentation. 